In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path

# Paths
DAILY_FOLDER = Path("data/daily")
AVERAGE_FILE = Path("data/averages/average_hot_scores.csv")
OUTPUT_FOLDER = Path("output/runtime")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

# Load latest daily CSV
daily_file = sorted(DAILY_FOLDER.glob("hot_stocks_*.csv"))[-1]
df_daily = pd.read_csv(daily_file)

# Load averages
df_avg = pd.read_csv(AVERAGE_FILE)

# Merge for comparison
df = df_daily.merge(df_avg[['symbol','HotScore']], on='symbol', how='left', suffixes=('_today','_avg'))

# Compute RuntimeScore (example)
df['RuntimeScore'] = 0.5*df['HotScore_today'] + 0.5*(df['HotScore_today']/df['HotScore_avg'])

# Filter top suggestions
top_runtime = df[df['RuntimeScore'] > df['RuntimeScore'].quantile(0.8)].sort_values('RuntimeScore', ascending=False)

# Save CSV with timestamp
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
csv_file = OUTPUT_FOLDER / f"runtime_suggestions_{timestamp}.csv"
top_runtime.to_csv(csv_file, index=False)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(12,6))
sns.barplot(x='symbol', y='RuntimeScore', data=top_runtime.head(20), palette='coolwarm', ax=ax)
plt.xticks(rotation=90)
plt.title("Top Runtime Suggestions")
png_file = OUTPUT_FOLDER / f"runtime_top20_{timestamp}.png"
plt.tight_layout()
plt.savefig(png_file)
plt.close(fig)


In [ ]:
import json

meta = {
    "timestamp": timestamp,
    "count": len(top_runtime),
    "top_symbols": top_runtime['symbol'].tolist(),
    "files": {
        "csv": str(csv_file),
        "chart": str(png_file)
    }
}

json_file = OUTPUT_FOLDER / f"metadata_runtime_{timestamp}.json"
with open(json_file, 'w') as f:
    json.dump(meta, f, indent=2)


In [ ]:

import firebase_admin
from firebase_admin import credentials, storage

def init_bucket():
    raw = os.environ.get("FIREBASE_SERVICE_ACCOUNT")
    if not raw:
        raise RuntimeError("FIREBASE_SERVICE_ACCOUNT empty")
    cred_info = json.loads(raw)
    cred = credentials.Certificate(cred_info)
    firebase_admin.initialize_app(cred, {"storageBucket":"tpmedici.appspot.com"})
    return storage.bucket()

def upload_folder_local(bucket, local_folder, remote_prefix="market/"):
    local_folder = Path(local_folder)
    if not local_folder.exists():
        print(f"Skip, folder not found: {local_folder}")
        return
    for f in local_folder.glob("**/*"):
        if f.is_file():
            rel = f.relative_to(local_folder)
            remote_path = f"{remote_prefix}{local_folder.name}/{rel.as_posix()}"
            blob = bucket.blob(remote_path)
            blob.upload_from_filename(str(f))
            print(f"Uploaded {f} -> {remote_path}")

bucket = init_bucket()
upload_folder_local(bucket, "output/runtime", remote_prefix="market/")
